In [ ]:
from pymongo import MongoClient
from bson.raw_bson import RawBSONDocument
from pymongoarrow.api import Schema
from datetime import datetime
from pymongoarrow.monkey import patch_all
import pandas as pd
import pyarrow
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import os
import geopandas as gpd
from shapely.geometry import Point
from matplotlib.colors import LinearSegmentedColormap, to_hex
import geojson
import datetime

mongo_connection = f"mongodb://"
database = "Impact_Analysis"
baseline_collection = "baseline" # insert Impact Analysis MASTER collection name here
dev_collection = "dev" # insert Impact Analysis DEV collection name here

baseline_color = 'darkblue'
baseline_label = 'Baseline'

dev_color = '#1f77b4'
dev_label = 'Dev'

custom_colors = [baseline_color, dev_color]
custom_labels = [baseline_label, dev_label]

## Functions for Graph Types

For this section, all you have to do it run the chunk of code. This would be something to copy and paste over to your notebook so that you are able to use these functions. The purpose of this is to keep the notebook looking clean! Another note, these functinos should be usable for all data. For example, there is a bar chart function that is used throughout the collection.

Functions included in the code block below are for: Bar charts, line charts, scatterplots, double bar charts, horizontal bar charts, pie charts, vehicle profile charts, and heatmaps.

### Dataset Comparisons

In [ ]:
def compare_datasets(main, *datasets, type_filter=None, custom_title=None, colors=None, dataset_names=None, orientation='horizontal', x_label='Count of Trips', y_label=None):

    """
    Plot the count of unique trip IDs in the main dataset and additional datasets.

    Parameters:
    - main: DataFrame or scalar, the main dataset containing unique trip IDs or a scalar value
    - *datasets: Variable number of DataFrames, additional datasets to compare with the main dataset
    - type_filter: str or None, optional, a filter based on the 'Type' column for the datasets
    - custom_title: str or None, optional, a custom title for the plot
    - colors: list of str or None, optional, colors for each dataset bar
    - dataset_names: list of str or None, optional, names for each dataset
    - orientation: str, optional, either 'horizontal' (default) or 'vertical'
    - x_label: str, optional, label for the x-axis
    - y_label: str or None, optional, label for the y-axis

    Returns:
    - None (displays the plot)
    """
    
    if isinstance(main, pd.DataFrame):
        main_count = main['UniqueTripID'].nunique()
    elif isinstance(main, (int, float)):
        main_count = main
    else:
        raise ValueError("Invalid type for 'main'. It should be a DataFrame or a scalar value.")

    if type_filter is not None:
        datasets = [df[df['Type'] == type_filter] if isinstance(df, pd.DataFrame) else df for df in datasets]

    main_count = main_count if isinstance(main, (int, float)) else main['UniqueTripID'].nunique()

    counts = [df['UniqueTripID'].nunique() if isinstance(df, pd.DataFrame) else df for df in datasets]

    main_count = round(main_count, 1)
    counts = [round(count, 1) if isinstance(count, (int, float)) else count for count in counts]

    if colors is None:
        colors = ['gray', 'red', 'blue', 'green']

    if dataset_names is None:
        dataset_names = ['Main'] + [f'Dataset {i}' for i in range(1, len(counts) + 1)]

    fig, ax = plt.subplots()

    if orientation == 'horizontal':
        bars = ax.barh(dataset_names, [main_count] + counts, color=colors[:len(dataset_names)])
        for bar, count in zip(bars, [main_count] + counts):
            plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f'{count}', va='center', ha='left')
        ax.grid(axis='x', linestyle='--', alpha=0.6)
        plt.xlabel(x_label)
        if y_label is not None:
            plt.ylabel(y_label)
    elif orientation == 'vertical':
        bars = ax.bar(dataset_names, [main_count] + counts, color=colors[:len(dataset_names)])
        for bar, count in zip(bars, [main_count] + counts):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{count}', va='bottom', ha='center')
        ax.grid(axis='y', linestyle='--', alpha=0.6)
        plt.ylabel(x_label)
        if y_label is not None:
            plt.xlabel(y_label)
    else:
        raise ValueError("Invalid value for 'orientation'. It should be 'horizontal' or 'vertical'.")

    if custom_title is not None:
        ax.set_title(custom_title)
    else:
        default_title = f'Count of Unique IDs in Each Dataset{" Filtered by Type: " + type_filter if type_filter else ""}'
        ax.set_title(default_title)

    plt.show()


### Histogram/Boxplot

In [ ]:
def histogram_boxplot(df, feature, title=None, event_types=None, figsize=(7, 7), kde=False, bins=None, boxplot_color="#f1f1f6", histogram_color="royalblue", threshold=None, legend_position='right'):
    """
    Create a combined histogram and boxplot for a given feature in the dataset.

    Parameters:
    - df: DataFrame, the input data frame containing the analysis data
    - feature: str, the column name for the feature to be analyzed
    - title: str or None, optional, the title of the plot (None for default title)
    - event_types: list or None, optional, a list of event types to include in the analysis (None for all event types)
    - figsize: tuple, optional, the size of the resulting plot (default is (7, 7))
    - boxplot_color: str, optional, the color of the boxplot (default is "#f1f1f6")
    - histogram_color: str, optional, the color of the histogram (default is "royalblue")
    - threshold: float or None, optional, a threshold value to filter out data points beyond the threshold (None for no filtering)
    - legend_position: str, optional, the position of the legend ('right' or 'left') (default is 'right')
    """
    if event_types:
        df = df[df['Type'].isin(event_types)]

    if threshold is not None:
        df = df[df[feature] <= threshold]

    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,
        sharex=True,
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )

    sns.boxplot(
        data=df, x=feature, ax=ax_box2, showmeans=True, color=boxplot_color
    ).set(title=f"Distribution of {title}" if title else f"Distribution of {feature}")

    if bins:
        sns.histplot(
            data=df, x=feature, kde=kde, ax=ax_hist2, bins=bins, color=histogram_color
        )
    else:
        sns.histplot(
            data=df, x=feature, kde=kde, ax=ax_hist2, color=histogram_color
        )

    mean_value = df[feature].mean()
    median_value = df[feature].copy().median()
    sum_value = df[feature].sum() 

    ax_hist2.axvline(
        mean_value, color="green", linestyle="--", label=f"Mean ({mean_value:.2f})"
    )
    ax_hist2.axvline(
        median_value, color="black", linestyle="-", label=f"Median ({median_value:.2f})"
    )

    total_ids = df['UniqueID'].nunique()

    ax_hist2.text(
        1.05,
        1.0,
        f"Total IDs: {total_ids}",
        transform=ax_hist2.transAxes,
        horizontalalignment='left',
        verticalalignment='top',
        bbox=dict(facecolor='#f1f1f6', edgecolor='gray', boxstyle='round'),
    )

    ax_hist2.text(
        1.05,
        0.90,
        f"Sum: {sum_value:.2f}",
        transform=ax_hist2.transAxes,
        horizontalalignment='left',
        verticalalignment='top',
        bbox=dict(facecolor='#f1f1f6', edgecolor='gray', boxstyle='round'),
    )
    
    if legend_position == 'right':
        legend_loc = 'upper left'
        bbox_anchor = (1.0, 0.85)  
    elif legend_position == 'left':
        legend_loc = 'upper right'
        bbox_anchor = (-0.3, 1.0)  
    else:
        raise ValueError("Invalid legend position. Use 'left' or 'right'.")

    ax_hist2.legend(loc=legend_loc, bbox_to_anchor=bbox_anchor)

### Remove Outliers

In [ ]:
def remove_outliers(df, column):
    """
    Remove outliers from a DataFrame based on a specified column using the IQR method.

    Parameters:
    - df: DataFrame, the input data frame containing the column with potential outliers
    - column: str, the name of the column to check for outliers

    Returns:
    - DataFrame, a filtered DataFrame excluding the outliers
    """

    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    df_filtered = df[(df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 * IQR)]

    return(df_filtered)

### Get Data Counts

In [ ]:
def gather_data_count(df, column):
    # break down all events and group by type
    columns = df[["UniqueID", column]]
    normalized_columns = columns.explode(column).dropna()
    df = normalized_columns[column].apply(pd.Series)
    grouped_columns = columns.join(df).dropna()

    grouped_column_count = grouped_columns.groupby('Type').count()
    return grouped_column_count

### Merge Event Data

In [ ]:
def merge_event_data(merge_column, event_column, baseline, dev):
    merged_data = baseline.merge(dev, on=merge_column, how='outer', suffixes=('_base', '_dev'))
    merged_data[[f'{event_column}_base', f'{event_column}_dev']] = merged_data[[f'{event_column}_base', f'{event_column}_dev']].fillna(0)

    return merged_data

### Bar Chart - Events/RTR

In [ ]:
def event_bar_chart(df, merge_column, event_column, event_types, legend_labels, title, x_label, y_label, include_types=None, types=None, figsize=(12, 9), legend_loc='upper right', colors=None):
    """
    Plot a horizontal bar graph for multiple event types.

    Parameters:
    - df: Merged and filtered DataFrame with event type columns.
    - event_types: List of event types to include in the plot.
    - include_types: List of event types to include in the plot (default is None, which includes all).
    - types: List of specific 'Type' values to include in the plot (default is None, which includes all).
    - figsize: Size of the figure.
    - legend_loc: Location of the legend in the plot.
    - colors: Dictionary mapping event types to colors.

    Returns:
    - None (displays the plot).
    """
    # If 'Type' is not in the columns, reset the index to make it a column
    if merge_column not in df.columns:
        df = df.reset_index()

    event_columns = [f'{event_column}_{etype}' for etype in event_types]

    if include_types is not None:
        df = df[df[merge_column].isin(include_types)]

    if types is not None:
        if isinstance(types, str):
            types = [types] 
        df = df[df[merge_column].isin(types)]

    valid_event_types = [etype for etype in event_types if f'{event_column}_{etype}' in df.columns]
    if not valid_event_types:
        print("No valid event types found in the DataFrame.")
        return

    total_column = f'Total_{", ".join(valid_event_types)}'
    df[total_column] = df[event_columns].sum(axis=1)
    df = df.sort_values(by=total_column, ascending=True)

    fig, ax = plt.subplots(figsize=figsize)

    bar_width = 0.2 

    for i, etype in enumerate(valid_event_types):
        column_name = f'{event_column}_{etype}'
        counts = df[column_name]
        positions = range(len(df))

        positions = [pos - i * bar_width for pos in positions]
        color = colors.get(etype, None) if colors else None
        ax.barh(positions, counts, height=bar_width, label=f'{etype.capitalize()}', color=color)
        for j, count in enumerate(counts):
            ax.text(int(count), positions[j], str(int(count)), ha='left', va='center', color='black')

    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_yticks(range(len(df)))
    ax.set_yticklabels(df[merge_column])
    ax.set_title(title)

    legend_labels = legend_labels 
    ax.legend(labels=legend_labels, loc=legend_loc)
    ax.grid(axis='x', linestyle='--', linewidth=0.5)

    plt.show()

### Merge RTR Data

In [ ]:
def gather_road_type_data(df):
    crtr = df[["UniqueTripID", "Analysis_Computed_RoadTypeReport"]]
    cnormalize_col = crtr.explode("Analysis_Computed_RoadTypeReport").dropna()
    adf = cnormalize_col['Analysis_Computed_RoadTypeReport'].apply(pd.Series)
    agrouped = crtr.join(adf).dropna()

    agrouped_by_class = agrouped.groupby(['UniqueTripID', 'Class']).agg({'DistMiles': 'sum'}).reset_index()

    total_distance = agrouped_by_class.groupby('UniqueTripID')['DistMiles'].sum().reset_index()
    agrouped_by_class = agrouped_by_class.merge(total_distance, on='UniqueTripID', suffixes=('', '_total'))
    agrouped_by_class['Percentage'] = (agrouped_by_class['DistMiles'] / agrouped_by_class['DistMiles_total']) * 100
    rtr_merged_df = agrouped.merge(agrouped_by_class[['UniqueTripID', 'Class', 'Percentage']], on=['UniqueTripID', 'Class'])
    rtr_merged_df = rtr_merged_df.drop('Analysis_Computed_RoadTypeReport', axis=1)
    artr_grouped_df = adf.groupby('Class').agg({'DistMiles': 'sum', 'DistMilesBuiltUp': 'sum'})
    artr_grouped_df = artr_grouped_df.reset_index()
    
    return artr_grouped_df

# Impact

## Get Data

### Analysis Schema & Projection

In [ ]:
default_schema = {

 }


In [ ]:
default_projection = {

 }

### Account Name Schema & Projection

In [ ]:
account_schema = {

}

In [ ]:
account_projection = {

}

### Baseline Impact Results - without change

In [ ]:
client = MongoClient(mongo_connection)
db1 = client[database]
collection1 = db1[baseline_collection] 
patch_all()

In [ ]:
master = collection1.aggregate_pandas_all([
    {"$project" : default_projection}
    ],
    schema = Schema(default_schema)
)

### Dev Impact Result - with change

In [ ]:
client = MongoClient(mongo_connection)
db2 = client[database]
collection2 = db2[dev_collection] 
patch_all()

In [ ]:
dev = collection2.aggregate_pandas_all([
    {"$sample" : {"size" : 10000}},
    {"$project" : default_projection}
    ],
    schema = Schema(default_schema)
)

### Accounts

In [ ]:
client = MongoClient(mongo_connection)
dbaccount = client["Accounts"]
accountcollection = dbaccount["accounts_map"]
patch_all()

In [ ]:
accountname = accountcollection.aggregate_pandas_all([
    {"$project" : account_projection}
    ],
    schema = Schema(account_schema)
)

## Data Prep

In [ ]:
impact_master = pd.DataFrame(master)
impact_dev = pd.DataFrame(dev)
accountname = pd.DataFrame(accountname)

accountname['AccountID'] = accountname['AccountID'].astype(int)

impact_master = impact_master.merge(accountname, left_on = "AccountID", right_on = "AccountID", how = "left")
impact_dev = impact_dev.merge(accountname, left_on = "AccountID", right_on = "AccountID", how = "left")

In [ ]:
impact_master = impact_master.drop_duplicates(subset=['UniqueID'])
impact_dev = impact_dev.drop_duplicates(subset=['UniqueID'])

In [ ]:
common_ids = set(impact_master['UniqueID']).intersection(impact_dev['UniqueID'])
impact_master = impact_master[impact_master['UniqueID'].isin(common_ids)]
impact_dev = impact_dev[impact_dev['UniqueID'].isin(common_ids)]

In [ ]:
len(impact_master)

In [ ]:
len(impact_dev)

## Distance (Miles)

### Plot Sum of Dist Miles - Bar Chart

In [ ]:
cSum_dev = impact_dev['Distance'].sum()
cSum = impact_master['Distance'].sum()

In [ ]:
compare_datasets(cSum, cSum_dev, custom_title='Sum of Mileages', colors=custom_colors, dataset_names=custom_labels, orientation='vertical',  x_label='Count of Miles', y_label='Branch')

### Plot Mean of Distance - Bar Chart

In [ ]:
cMean_dev = impact_dev['Distance'].mean()
cMean = impact_master['Distance'].mean()

In [ ]:
compare_datasets(cMean, cMean_dev, custom_title='Average of Mileages', colors=custom_colors, dataset_names=custom_labels, orientation='vertical',  x_label='Average of Miles', y_label='Branch')

## Mileage Diff

### (EDIT) Create .csv to Investigate

In [ ]:
mileage_master = impact_master[['UniqueID','DistMiles']]
mileage_dev = impact_dev[['UniqueTripID','DistMiles']]
mileage_combined = pd.merge(mileage_master, mileage_dev, on='UniqueID', suffixes=('_master', '_dev'))
mileage_combined['Mileage_diff'] = abs(mileage_combined['DistMiles'] - mileage_combined['DistMiles'])
mileage_combined = mileage_combined.sort_values(by='Mileage_diff', ascending=False)

# mileage_combined.to_csv('all.csv', index=False) # Edit name of file to be output

### Boxplot Histogram for Mileage Diff

In [ ]:
histogram_boxplot(mileage_combined, 'Mileage_diff')

## Estimated Time Taken (ETT)

### Convert ETTSeconds to HH:MM:SS

In [ ]:
def convert_seconds(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return hours, minutes, seconds

### Plot Sum of ETT

In [ ]:
cETTSum_dev = impact_dev['Analysis_Computed_ETTSeconds'].sum()
cETTSum = impact_master['Analysis_Computed_ETTSeconds'].sum()

In [ ]:
compare_datasets(cETTSum, cETTSum_dev, custom_title='Sum of ETTSeconds', colors=custom_colors, dataset_names=custom_labels, orientation='vertical',  x_label='ETT', y_label='Branch')

### Plot Average of ETT

In [ ]:
cETTMean_dev = impact_dev['ETTS'].mean()
cETTMean = impact_master['ETTS'].mean()

In [ ]:
compare_datasets(cETTMean, cETTMean_dev, custom_title='Average of ETTSeconds', colors=custom_colors, dataset_names=custom_labels, orientation='vertical',  x_label='ETT', y_label='Branch')

## Events

### Legal

In [ ]:
baseline_legal_type_counts = gather_data_count(impact_master, 'Legal')
dev_legal_type_counts = gather_data_count(impact_dev, 'Legal')

merged_legal = merge_event_data('Type', 'Legal', baseline_legal_type_counts, dev_legal_type_counts)

In [ ]:
event_bar_chart(merged_legal, 'Type', 'Legal', event_types=['base', 'dev'], legend_labels = ['Baseline Legal', 'Dev Legal'], title = 'Counts of Legal Events', x_label='Count', y_label='Event Type', colors={'base': baseline_color, 'dev': dev_color}, legend_loc='lower right')

## Road Type Report (RTR)

In [ ]:
rtr_baseline = gather_road_type_data(impact_master)
rtr_dev = gather_road_type_data(impact_dev)

rtr_merged = merge_event_data('Class', 'DistMiles', rtr_baseline, rtr_dev)

In [ ]:
event_bar_chart(rtr_merged, 'Class', 'DistMiles', event_types=['base', 'dev'], legend_labels = ['Baseline', 'Dev'], title = 'Total Mileage by Road Class', x_label='Mileage', y_label='Road Class', colors={'base': baseline_color, 'dev': dev_color}, legend_loc='lower right')